> First time use: follow instructions in the README.md file in this directory.

# Network analysis of the Jesuit Mission in China

In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.26
Project name: dehergne
Project home: /Users/jrc/mhk-home/sources/dehergne
Database type: sqlite
Database name: dehergne
Kleio image: timelinkserver/kleio-server
Kleio server token: fQOU0...
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/mhk-home/sources/dehergne
Kleio server container: friendly_elion
Kleio version requested: latest
Kleio server version: 12.9.588 (2025-06-06 16:04:22)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne, project_home=/Users/jrc/mhk-home/sources/dehergne, db_type=sqlite, db_name=dehergne, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


In [2]:
# generate network of those on the same voyage
from timelink.networks import network_from_attribute
from timelink.pandas import attribute_values

attribute = 'wicky-viagem'  # attribute to use for the network

attribute_values_df = attribute_values(attribute,db=tlnb.db)
attribute_values_df

count  date_min  date_max
value                           
57        31  16180400  16180400
87        20  16550323  16550323
89        20  16570404  16570404
99        18  16730315  16730315
117       16      1694  16940000
...      ...       ...       ...
81         1  16480408  16480408
83         1  16500420  16500420
86         1  16540324  16540324
9          1  15560330  15560330
98         1  16720301  16720301

[136 rows x 3 columns]

In [3]:
wg = network_from_attribute(attribute='wicky-viagem',mode='value-node',db=tlnb.db)
[(m,wg.nodes[m]) for m in list(wg.nodes)[0:3]]

[('57', {'id': '57', 'desc': '57', 'type': 'wicky-viagem'}),
 ('deh-mateus-gago',
  {'desc': 'Mateus Gago',
   'id': 'deh-mateus-gago',
   'type': 'person',
   'group': 'n',
   'date': '1618-04',
   'source': 'dehergne-g'}),
 ('deh-giovanni-domenico-gayati',
  {'desc': 'Giovanni Domenico Gayati',
   'id': 'deh-giovanni-domenico-gayati',
   'type': 'person',
   'group': 'n',
   'date': '1618-04',
   'source': 'dehergne-g'})]

In [4]:
from networkx.drawing.nx_agraph import write_dot

write_dot(wg,'wicki-viagens.dot')

In [26]:
# generate network of those on the same college
jc = network_from_attribute('jesuita-entrada', mode='value-node',db=tlnb.db)
write_dot(jc,'jesuitas-entrada-colegios.dot')

In [8]:
[jc.nodes[m] for m in list(jc.nodes)[0:6]]

[{'id': 'Coimbra', 'desc': 'Coimbra', 'type': 'jesuita-entrada'},
 {'desc': 'Pedro de Alcáçova',
  'id': 'deh-pedro-de-alcacova',
  'type': 'person',
  'group': 'n',
  'date': '1548-05-02',
  'source': 'dehergne-a'},
 {'type': 'year', 'desc': '1542'},
 {'desc': 'Belchior Nunes Barreto',
  'id': 'deh-belchior-nunes-barreto',
  'type': 'person',
  'group': 'n',
  'date': '1543-03-11',
  'source': 'dehergne-n'},
 {'type': 'year', 'desc': '1543'},
 {'desc': 'Belchior Miguel Carneiro Leitão',
  'id': 'deh-belchior-miguel-carneiro-leitao',
  'type': 'person',
  'group': 'n',
  'date': '1543-04-25',
  'source': 'dehergne-c'}]

## Visualizar

In [ ]:

from timelink.networks import draw_network

# Ensure that the graph is in G
G=wg

#Choose a title!
title = 'jesuit_travelling_network'
tooltips = [("desc","@desc"),("type","@type"), ("date","@date")]

node_color_attr = 'type'
node_types = {G.nodes[n][node_color_attr] for n in G.nodes}
print(f"Node types: {node_types}")
node_colors = {'person':'green', 'wicky-viagem':'red'}


draw_network(G,
            title=title,
            node_color_attr=node_color_attr,
            node_colors=node_colors,
            node_tooltips=tooltips,
            iterations=100,
            width=800,
            scale=1.5,
            height=800,)




Node types: {'person', 'wicky-viagem'}


Loading BokehJS ...

In [27]:
from timelink.networks import draw_network
# Ensure that the graph is in G
G=jc

#Choose a title!
title = 'jesuit_entry_college'

tooltips = [("desc","@desc"),
            ("type","@type"),
            ("original_id","@original_id"),
            ("date","@date")]

node_color_attr = 'type'
# find nodes with no type attribute
for n in G.nodes:
    if 'type' not in G.nodes[n]:
        print(f"Node {n} has no type attribute: {G.nodes[n]}")
node_types = {G.nodes[n][node_color_attr] for n in G.nodes}
print(f"Node types: {node_types}")
node_colors = {'person':'green', 'jesuita-entrada':'blue'}


draw_network(G,
            title=title,
            node_color_attr=node_color_attr,
            node_colors=node_colors,
            node_tooltips=tooltips,
            width=800,
            height=800)



Node types: {'jesuita-entrada', 'person'}


Loading BokehJS ...

## Join both networks

In [32]:
import networkx as nx
from networkx import MultiGraph

jesuit_net = MultiGraph()

jesuit_net=nx.compose(wg,jc)

write_dot(jesuit_net,'college_and_voyage.dot')

In [33]:
#Choose a title!
title = 'college_and_voyage'

# Ensure that the graph is in G
G=jesuit_net

tooltips = [("desc","@desc"),
            ("type","@type"),
            ("original_id","@id"),
            ("date", "@date")]
node_types = {G.nodes[n][node_color_attr] for n in G.nodes}
print(f"Node types: {node_types}")
node_colors = {'person':'green', 'jesuita-entrada':'blue', 'wicky-viagem':'red'}
draw_network(G,
            title=title,
            node_color_attr='type',
            node_colors=node_colors,
            node_tooltips=tooltips,
            iterations=100,
            height=800,  # Adjust height and width as needed
            width=800,)




Node types: {'jesuita-entrada', 'person', 'wicky-viagem'}


Loading BokehJS ...

## Places of stay

In [37]:
from timelink.pandas import attribute_values

places_of_stay = attribute_values('estadia@wikidata',db=tlnb.db)
places_of_stay

count  date_min  date_max
value                                                           
https://www.wikidata.org/wiki/Q14773     329         0  17730000
https://www.wikidata.org/wiki/Q956       191         0  17910000
https://www.wikidata.org/wiki/Q16572     164         0  17850000
https://www.wikidata.org/wiki/Q1171      126         0  17860123
https://www.wikidata.org/wiki/Q597        88         0  17810300
...                                      ...       ...       ...
https://www.wikidata.org/wiki/Q8818        1  17080908  17080908
https://www.wikidata.org/wiki/Q889         1  17000000  17000000
https://www.wikidata.org/wiki/Q916         1  16810000  16810000
https://www.wikidata.org/wiki/Q989892      1  17051017  17051017
https://www.wikidata.org/wiki/Q99          1         0         0

[383 rows x 3 columns]

In [34]:
import requests
import time

estadias = network_from_attribute('estadia@wikidata',mode='value-node',db=tlnb.db)
for node in estadias.nodes:
    if estadias.nodes[node].get('type') == 'estadia@wikidata' and 'id' in estadias.nodes[node]:
        wikidata_url = estadias.nodes[node]['id']
        wikidata_id = wikidata_url.split('/')[-1]
        time.sleep(0.5)
        response = requests.get(f"https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json")
        if response.status_code == 200:
            data = response.json()
            if 'entities' in data and wikidata_id in data['entities']:
                entity = data['entities'][wikidata_id]
                if 'labels' in entity and 'en' in entity['labels']:
                    estadias.nodes[node]['desc'] = entity['labels']['en']['value']
                    print(f"Node {node} updated with description: {estadias.nodes[node]['desc']}")
[estadias.nodes[m] for m in list(estadias.nodes)[0:3]]

Node https://www.wikidata.org/wiki/Q14773 updated with description: Macau
Node https://www.wikidata.org/wiki/Q956 updated with description: Beijing
Node https://www.wikidata.org/wiki/Q16572 updated with description: Guangzhou
Node https://www.wikidata.org/wiki/Q1171 updated with description: Goa
Node https://www.wikidata.org/wiki/Q597 updated with description: Lisbon
Node https://www.wikidata.org/wiki/Q148 updated with description: People's Republic of China
Node https://www.wikidata.org/wiki/Q16666 updated with description: Nanjing
Node https://www.wikidata.org/wiki/Q220 updated with description: Rome
Node https://www.wikidata.org/wiki/Q622778 updated with description: Tonkin
Node https://www.wikidata.org/wiki/Q8686 updated with description: Shanghai
Node https://www.wikidata.org/wiki/Q869 updated with description: Thailand
Node https://www.wikidata.org/wiki/Q2235155 updated with description: Huguang
Node https://www.wikidata.org/wiki/Q71724 updated with description: Lorient
Node http

[{'id': 'https://www.wikidata.org/wiki/Q14773',
  'desc': 'Macau',
  'type': 'estadia@wikidata'},
 {'desc': 'Philippe Gonzaga',
  'id': 'deh-philippe-gonzaga',
  'type': 'person',
  'group': 'n',
  'date': '1758-03-16',
  'source': 'dehergne-g'},
 {'desc': 'Giampaolo Gozani',
  'id': 'deh-giampaolo-gozani',
  'type': 'person',
  'group': 'n',
  'date': '1722',
  'source': 'dehergne-g'}]

In [36]:
# visualize
title = 'place of stay'

# Ensure that the graph is in G
G=estadias

tooltips = [("desc","@desc"),
            ("type","@type"),
            ("original_id","@id"),
            ("date", "@date")]
node_types = {G.nodes[n][node_color_attr] for n in G.nodes}
print(f"Node types: {node_types}")
node_colors = {'person':'green', 'jesuita-entrada':'blue', 'wicky-viagem':'red', 'estadia@wikidata':'yellow'}
draw_network(G,
            title=title,
            node_color_attr='type',
            node_colors=node_colors,
            node_tooltips=tooltips,
            iterations=300,
            height=800,  # Adjust height and width as needed
            width=800,)


Node types: {'person', 'estadia@wikidata'}


Loading BokehJS ...